<a href="https://colab.research.google.com/github/eyemveda/smc-bot-/blob/main/Backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import re
from backtesting import Backtest, Strategy

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
# Read the file with tab separator, skipping the first row
df_raw = pd.read_csv("/content/XAUUSDm_M1_2025.csv", sep='\t', engine='python', header=None, skiprows=1)

# Drop empty columns caused by separator - this might not be needed with correct sep
# df_raw = df_raw.dropna(axis=1, how='all')

# Rename columns manually - assuming the tab separation results in 9 columns
df_raw.columns = ["DATE", "TIME", "OPEN", "HIGH", "LOW", "CLOSE", "TICKVOL", "VOL", "SPREAD"]

# Combine DATE and TIME into a single datetime column
df_raw["DateTime"] = pd.to_datetime(df_raw["DATE"] + " " + df_raw["TIME"])

# Set index and clean up
df_1m = df_raw.set_index("DateTime")
df_1m = df_1m.rename(columns=str.lower)
# Select relevant columns, making sure they are in lowercase after renaming
df_1m = df_1m[['open', 'high', 'low', 'close', 'vol']]

In [ ]:
# Optimizing for multitimeframe analysis
def resample_to_tf(df, timeframe):
    return df.resample(timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'vol': 'sum' # Changed 'volume' to 'vol'
    }).dropna()

df_5m = resample_to_tf(df_1m, "5min") # Changed "5T" to "5min"
df_15m = resample_to_tf(df_1m, "15min") # Changed "15T" to "15min"
df_1h = resample_to_tf(df_1m, "1H")
df_4h = resample_to_tf(df_1m, "4H")

/tmp/ipython-input-2842005807.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return df.resample(timeframe).agg({


In [ ]:
# Fixing the Warning In the Above Cell
# Fixing the Warning
def normalize_tf(tf: str) -> str:
    import re

    if not isinstance(tf, str):
        return tf
    s = tf.strip().lower()
    m = re.match(r'^(\d+)\s*([a-zA-Z]+)$', s)
    if not m:
        return s  # leave as-is, let pandas raise if invalid

    n, unit = m.groups()

    # Map to pandas-friendly units (lowercase for everything except 'M' for months)
    unit_map = {
        't': 'min', 't.': 'min', 'min': 'min', 'm': 'min', 'minute': 'min', 'minutes': 'min',
        'h': 'h', 'hour': 'h', 'hours': 'h',
        'd': 'd', 'day': 'd', 'days': 'd',
        'w': 'w', 'week': 'w', 'weeks': 'w',
        'mo': 'M', 'month': 'M', 'months': 'M'
    }

    unit_norm = unit_map.get(unit, unit)
    return f"{n}{unit_norm}"


In [ ]:
# atr stuff

def atr(df, period=14):
    """
    Average True Range (ATR) for backtesting & live trading.
    Uses Welles Wilder's method with proper shift to avoid lookahead.
    """
    if df is None or len(df) < 2:
        return pd.Series(dtype=float)

    high = df['high']
    low = df['low']
    close = df['close']

    tr = pd.concat([
        (high - low).abs(),
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)

    # Wilder's smoothing (EMA-like) for classic ATR
    atr_values = tr.ewm(alpha=1/period, adjust=False).mean()

    # Shift so the ATR at bar X only uses info up to bar X-1
    atr_values = atr_values.shift(1)

    return atr_values

# Liquidity Sweep Detection Logic

In [ ]:
def resample_ohlcv(df_1m, timeframe):
    tf = normalize_tf(timeframe)
    agg = {'open':'first','high':'max','low':'min','close':'last'}
    if 'volume' in df_1m.columns: agg['volume']='sum'
    res = df_1m.resample(tf).agg(agg)
    res = res.dropna(subset=['open','high','low','close'])
    return res

def atr_series(df, n=14):
    h,l,c = df['high'], df['low'], df['close']
    tr = pd.concat([h-l, (h-c.shift()).abs(), (l-c.shift()).abs()], axis=1).max(axis=1)
    return tr.ewm(span=n, adjust=False).mean()

def zigzag_pivots_from_df(df_tf, pct=0.004, use_atr=False, atr_period=14):
    if df_tf.empty:
        return []
    closes = df_tf['close'].values
    idx = list(df_tf.index)
    pivots = []
    last_price = closes[0]
    last_idx = 0
    direction = None
    atr = atr_series(df_tf, n=atr_period).reindex(df_tf.index).bfill().values if use_atr else None

    for i in range(1, len(closes)):
        threshold = (atr[i] if use_atr else closes[i] * pct)
        if threshold <= 0:
            continue
        change = closes[i] - last_price
        if direction is None:
            if abs(change) >= threshold:
                direction = 'up' if change > 0 else 'down'
                pivots.append((idx[last_idx], float(last_price), 'low' if direction=='up' else 'high'))
                last_price = closes[i]; last_idx = i
        elif direction == 'up':
            if closes[i] > last_price:
                last_price = closes[i]; last_idx = i
            elif (last_price - closes[i]) >= threshold:
                pivots.append((idx[last_idx], float(last_price), 'high'))
                direction = 'down'
                last_price = closes[i]; last_idx = i
        else:  # down
            if closes[i] < last_price:
                last_price = closes[i]; last_idx = i
            elif (closes[i] - last_price) >= threshold:
                pivots.append((idx[last_idx], float(last_price), 'low'))
                direction = 'up'
                last_price = closes[i]; last_idx = i
    return pivots

def is_level_swept(level_price, level_ts, direction, df_1m, tol_abs=0.0):
    mask = df_1m.index > pd.Timestamp(level_ts)
    if mask.sum() == 0:
        return False
    sub = df_1m.loc[mask]
    if direction == 'high':
        return (sub['high'] >= (level_price + tol_abs)).any()
    else:
        return (sub['low'] <= (level_price - tol_abs)).any()

def cluster_levels(levels, tol):
    if not levels:
        return []
    levels = sorted(levels)
    clusters=[]
    for p in levels:
        if not clusters:
            clusters.append([p])
        else:
            if abs(p - np.mean(clusters[-1])) <= tol:
                clusters[-1].append(p)
            else:
                clusters.append([p])
    return [{'level': float(np.mean(c)), 'count': len(c)} for c in clusters]

def build_mtf_liquidity_debug(df_1m,
                        use_tfs = ['4h','1h'],
                        include_15m_for_tp=True,
                        zigzag_pct_val = 0.003,   # start slightly lower
                        zigzag_use_atr = True,
                        atr_period = 14,
                        equal_tol_atr_mult=0.30,
                        verbose=True):
    atr_1m = atr_series(df_1m, n=atr_period)
    median_atr = float(atr_1m.median())
    tol = max(median_atr * equal_tol_atr_mult, 1e-6)

    all_pivots = []
    debug = {'resampled_rows':{}, 'raw_pivots_per_tf':{}}

    for tf in use_tfs:
        df_tf = resample_ohlcv(df_1m, tf)
        debug['resampled_rows'][tf] = len(df_tf)
        piv = zigzag_pivots_from_df(df_tf, pct=zigzag_pct_val, use_atr=zigzag_use_atr, atr_period=atr_period)
        debug['raw_pivots_per_tf'][tf] = len(piv)
        all_pivots += [(ts, price, typ, tf) for (ts, price, typ) in piv]

    if verbose:
        print("median ATR (1m):", median_atr)
        print("clustering tol:", tol)
        for tf in use_tfs:
            print(f"Resampled {tf}: {debug['resampled_rows'].get(tf,0)} rows, pivots found: {debug['raw_pivots_per_tf'].get(tf,0)}")

    pivot_rows = []
    for ts, price, typ, tf in all_pivots:
        swept = is_level_swept(price, ts, typ, df_1m, tol_abs=tol*0.5)
        pivot_rows.append({'ts':pd.Timestamp(ts),'price':float(price),'type':typ,'tf':tf,'swept':swept})
    pivots_df = pd.DataFrame(pivot_rows).sort_values(['tf','ts'], ascending=[True,True]).reset_index(drop=True)

    if verbose:
        print("total pivots (all TFs):", len(pivots_df))
        if not pivots_df.empty:
            print("swept count:", pivots_df['swept'].sum(), "unswept:", (~pivots_df['swept']).sum())

    pivots_unswept = pivots_df[~pivots_df['swept']].copy()
    high_prices = list(pivots_unswept[pivots_unswept['type']=='high']['price'])
    low_prices  = list(pivots_unswept[pivots_unswept['type']=='low']['price'])
    high_clusters = cluster_levels(high_prices, tol)
    low_clusters  = cluster_levels(low_prices, tol)

    rows=[]
    for c in high_clusters:
        rows.append({'level':c['level'],'side':'sell','count':c['count'],'source':'equal_highs','strength':1 + c['count']/2})
    for c in low_clusters:
        rows.append({'level':c['level'],'side':'buy','count':c['count'],'source':'equal_lows','strength':1 + c['count']/2})

    liquidity_df = pd.DataFrame(rows).sort_values('strength', ascending=False).reset_index(drop=True) if rows else pd.DataFrame(columns=['level','side','count','source','strength'])

    # 15min TP pivots
    tp_15m = pd.DataFrame()
    if include_15m_for_tp:
        df_15m = resample_ohlcv(df_1m, '15min')
        piv15 = zigzag_pivots_from_df(df_15m, pct=zigzag_pct_val/2, use_atr=zigzag_use_atr, atr_period=atr_period)
        rows15=[]
        for ts,price,typ in piv15:
            swept = is_level_swept(price, ts, typ, df_1m, tol_abs=tol*0.5)
            if not swept:
                rows15.append({'ts':pd.Timestamp(ts),'price':price,'type':typ})
        if rows15:
            tp_15m = pd.DataFrame(rows15)
    return liquidity_df, tp_15m, pivots_df, pivots_unswept, debug

# Fairvalue Gap Detection


In [ ]:
  # ---------------- Fair Value Gap (FVG) Detection ----------------
def identify_fvg(df, min_gap_pct=0.0, use_atr_filter=False, atr_period=14, atr_mult=0.5, tol_abs=None):
    """
    Identify bullish/bearish Fair Value Gaps (FVGs) and optionally filter by size/ATR or sweep status.

    Returns:
        DataFrame with columns:
        - fvg_type: 1 = bullish, -1 = bearish, 0 = none
        - fvg_top, fvg_bottom: FVG boundaries
        - fvg_mid: midpoint of the gap
        - fvg_range: size of the gap
    """
    df = df.copy()

    if len(df) < 3:
        df[['fvg_type', 'fvg_top', 'fvg_bottom', 'fvg_mid', 'fvg_range']] = np.nan
        df['fvg_type'] = 0
        return df

    # Optional ATR for filtering
    atr_vals = atr(df, period=atr_period) if use_atr_filter else None

    bullish = (df['low'] > df['high'].shift(2))
    bearish = (df['high'] < df['low'].shift(2))

    df['fvg_type'] = 0
    df[['fvg_top', 'fvg_bottom', 'fvg_mid', 'fvg_range']] = np.nan

    # --- Bullish FVG ---
    for idx in df[bullish].index:
        gap_size = df.at[idx, 'low'] - df.at[idx - 2, 'high']
        if min_gap_pct > 0 and (gap_size / df.at[idx, 'low']) < min_gap_pct:
            continue
        if use_atr_filter and gap_size < atr_vals.at[idx] * atr_mult:
            continue
        # Sweep check: has price closed through the gap?
        if tol_abs is not None:
            swept = (df.loc[idx:, 'low'] <= df.at[idx - 2, 'high'] + tol_abs).any()
            if swept:
                continue
        df.at[idx, 'fvg_type'] = 1
        df.at[idx, 'fvg_bottom'] = df.at[idx - 2, 'high']
        df.at[idx, 'fvg_top'] = df.at[idx, 'low']
        df.at[idx, 'fvg_mid'] = (df.at[idx, 'low'] + df.at[idx - 2, 'high']) / 2
        df.at[idx, 'fvg_range'] = gap_size

    # --- Bearish FVG ---
    for idx in df[bearish].index:
        gap_size = df.at[idx - 2, 'low'] - df.at[idx, 'high']
        if min_gap_pct > 0 and (gap_size / df.at[idx, 'high']) < min_gap_pct:
            continue
        if use_atr_filter and gap_size < atr_vals.at[idx] * atr_mult:
            continue
        if tol_abs is not None:
            swept = (df.loc[idx:, 'high'] >= df.at[idx - 2, 'low'] - tol_abs).any()
            if swept:
                continue
        df.at[idx, 'fvg_type'] = -1
        df.at[idx, 'fvg_bottom'] = df.at[idx - 2, 'low']
        df.at[idx, 'fvg_top'] = df.at[idx, 'high']
        df.at[idx, 'fvg_mid'] = (df.at[idx, 'high'] + df.at[idx - 2, 'low']) / 2
        df.at[idx, 'fvg_range'] = gap_size

    return df


# Retracements In FVG

In [ ]:
def track_fvg_fills(df, fvg_col_type='fvg_type', top_col='fvg_top', bottom_col='fvg_bottom'):
    """
    Tracks whether Fair Value Gaps (FVGs) have been retraced/filled.

    Parameters:
        df (pd.DataFrame): Must have columns [fvg_type, fvg_top, fvg_bottom]
        fvg_col_type: column name for FVG type (1 = bullish, -1 = bearish)
        top_col: top boundary of gap
        bottom_col: bottom boundary of gap

    Returns:
        pd.DataFrame: same DF with new 'fvg_filled' (bool) column
    """
    df = df.copy()
    df['fvg_filled'] = False  # default

    active_fvgs = []  # store (index, type, top, bottom)

    for idx, row in df.iterrows():
        #  Add new FVG if found
        if row[fvg_col_type] != 0 and not np.isnan(row[top_col]) and not np.isnan(row[bottom_col]):
            active_fvgs.append({
                'created_at': idx,
                'type': row[fvg_col_type],
                'top': row[top_col],
                'bottom': row[bottom_col],
                'filled': False
            })

        #  Check active FVGs for retrace/fill
        for fvg in active_fvgs:
            if fvg['filled']:
                continue  # already filled, skip

            if fvg['type'] == 1:  # bullish FVG
                if row['low'] <= fvg['top']:
                    fvg['filled'] = True

            elif fvg['type'] == -1:  # bearish FVG
                if row['high'] >= fvg['bottom']:
                    fvg['filled'] = True

        # Mark fills in df
        for fvg in active_fvgs:
            if fvg['filled'] and idx >= fvg['created_at']:
                df.at[idx, 'fvg_filled'] = True

    return df


# Inverse Fair Value Gaps (IFVGs)

In [ ]:
def identify_ifvg(df, reference_break_time=None, min_gap_pct=0.0, use_atr_filter=False, atr_period=14, atr_mult=0.5):
    """
    Identify inverse Fair Value Gaps (iFVGs) — FVGs created before a break of structure,
    which remain unmitigated until after the break.

    Parameters:
        df : DataFrame with OHLC
        reference_break_time : optional pd.Timestamp, FVG must occur before this time
        min_gap_pct : minimum gap size as fraction of price to be considered
        use_atr_filter : whether to filter by ATR size
        atr_period : ATR period if ATR filter is used
        atr_mult : ATR multiplier for filtering
    """
    df = identify_fvg(df.copy(), min_gap_pct=min_gap_pct, use_atr_filter=use_atr_filter,
                      atr_period=atr_period, atr_mult=atr_mult)

    if 'fvg_type' not in df.columns:
        return pd.DataFrame()

    # Filter to actual FVGs
    fvg_candidates = df[df['fvg_type'] != 0].copy()
    if fvg_candidates.empty:
        return pd.DataFrame()

    fvg_candidates['mitigated'] = False
    fvg_candidates['mitigated_at'] = pd.NaT

    index_list = list(df.index)  # avoid .get_loc in loop

    for i, idx in enumerate(index_list):
        if idx not in fvg_candidates.index:
            continue

        ob_top = fvg_candidates.at[idx, 'fvg_top']
        ob_bottom = fvg_candidates.at[idx, 'fvg_bottom']
        fvg_type = fvg_candidates.at[idx, 'fvg_type']

        # Check for mitigation in future bars
        for j in range(i+1, len(index_list)):
            c = df.iloc[j]
            if pd.isna(ob_top) or pd.isna(ob_bottom):
                break

            if fvg_type == 1:  # bullish
                if c['low'] <= ob_top:
                    fvg_candidates.at[idx, 'mitigated'] = True
                    fvg_candidates.at[idx, 'mitigated_at'] = index_list[j]
                    break
            elif fvg_type == -1:  # bearish
                if c['high'] >= ob_bottom:
                    fvg_candidates.at[idx, 'mitigated'] = True
                    fvg_candidates.at[idx, 'mitigated_at'] = index_list[j]
                    break

    # Keep unmitigated before break time (if provided)
    if reference_break_time is not None:
        if isinstance(reference_break_time, (pd.Timestamp, datetime)):
            i_fvg = fvg_candidates[
                (~fvg_candidates['mitigated']) &
                (fvg_candidates.index < reference_break_time)
            ]
        else:
            i_fvg = fvg_candidates[~fvg_candidates['mitigated']]
    else:
        i_fvg = fvg_candidates[~fvg_candidates['mitigated']]

    return i_fvg


# Order Block

In [ ]:
def identify_order_blocks(df, atr_period=14, atr_mult=0.5, use_atr_filter=False):
    """
    Identify bullish and bearish Order Blocks independently.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns ['open', 'high', 'low', 'close'].
    atr_period : int, optional
        ATR lookback period for displacement filter (default=14).
    atr_mult : float, optional
        Multiplier for ATR filter (default=0.5).
    use_atr_filter : bool, optional
        If True, only accept OBs with displacement > ATR * atr_mult.

    Returns
    -------
    pd.DataFrame
        Original DataFrame with added columns:
        - ob_type : int (1 = bullish OB, -1 = bearish OB, 0 = none)
        - ob_top : float (high of the OB candle)
        - ob_bottom : float (low of the OB candle)
    """
    df = df.copy()
    df['ob_type'] = 0
    df['ob_top'] = np.nan
    df['ob_bottom'] = np.nan

    # Calculate ATR if filter enabled
    if use_atr_filter:
        hl_range = df['high'] - df['low']
        hc_range = (df['high'] - df['close'].shift()).abs()
        lc_range = (df['low'] - df['close'].shift()).abs()
        tr = pd.concat([hl_range, hc_range, lc_range], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=atr_period, min_periods=1).mean()
    else:
        df['atr'] = np.nan

    if len(df) < 4:
        return df

    # Loop through bars to detect OBs
    for i in range(3, len(df) - 1):
        cur = df.iloc[i]
        prev = df.iloc[i - 1]
        ob_candle = df.iloc[i - 2]
        ob_prev_candle = df.iloc[i - 3]

        # Skip if any key candle data is missing
        if pd.isna(ob_prev_candle['low']) or pd.isna(ob_prev_candle['high']):
            continue

        # Price displacement logic
        bullish_displacement = (cur['close'] > cur['open']) and (cur['close'] > prev['high'])
        bearish_displacement = (cur['close'] < cur['open']) and (cur['close'] < prev['low'])

        # ATR filter for displacement
        if use_atr_filter:
            move_size = abs(cur['close'] - cur['open'])
            if move_size < (cur['atr'] * atr_mult):
                continue

        # Bullish OB: last down candle before strong bullish move & liquidity sweep
        if bullish_displacement and (ob_candle['close'] < ob_candle['open']):
            swept_liquidity = ob_candle['low'] < ob_prev_candle['low']
            if swept_liquidity:
                df.at[i - 2, 'ob_type'] = 1
                df.at[i - 2, 'ob_top'] = ob_candle['high']
                df.at[i - 2, 'ob_bottom'] = ob_candle['low']

        # Bearish OB: last up candle before strong bearish move & liquidity sweep
        elif bearish_displacement and (ob_candle['close'] > ob_candle['open']):
            swept_liquidity = ob_candle['high'] > ob_prev_candle['high']
            if swept_liquidity:
                df.at[i - 2, 'ob_type'] = -1
                df.at[i - 2, 'ob_top'] = ob_candle['high']
                df.at[i - 2, 'ob_bottom'] = ob_candle['low']

    return df


In [ ]:
# Breaker Block

def identify_breaker_blocks(df,
                             sweep_lookahead=10,
                             break_lookahead=30,
                             atr_period=14,
                             atr_mult=0.5,
                             use_atr_filter=False):
    """
    Identify breaker blocks from price action.

    Breaker Block Definition:
      - Starts as an Order Block (OB)
      - Liquidity sweep (price pierces OB level)
      - Then price reverses and breaks structure in the opposite direction.

    Parameters:
        df (pd.DataFrame): Must have ['open','high','low','close'].
        sweep_lookahead (int): Bars after OB to check for liquidity sweep.
        break_lookahead (int): Bars after sweep to check for structure break.
        atr_period (int): ATR period for OB filter.
        atr_mult (float): Min displacement relative to ATR.
        use_atr_filter (bool): If True, filter weak OB moves.

    Returns:
        pd.DataFrame: Breaker blocks with columns:
                      ['ob_index', 'ob_type', 'ob_top', 'ob_bottom', 'breaker_detected_at']
    """
    df = df.copy()

    # -------------------
    # Internal OB detection
    # -------------------
    df['ob_type'] = 0
    df['ob_top'] = np.nan
    df['ob_bottom'] = np.nan

    if use_atr_filter:
        high_low = df['high'] - df['low']
        high_close = np.abs(df['high'] - df['close'].shift())
        low_close = np.abs(df['low'] - df['close'].shift())
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=atr_period, min_periods=1).mean()
    else:
        df['atr'] = np.nan

    for i in range(3, len(df) - 1):
        cur = df.iloc[i]
        prev = df.iloc[i - 1]
        ob = df.iloc[i - 2]
        ob_prev = df.iloc[i - 3]

        is_bull_move = (cur['close'] > cur['open']) and (cur['close'] > prev['high'])
        is_bear_move = (cur['close'] < cur['open']) and (cur['close'] < prev['low'])

        if use_atr_filter:
            move_size = abs(cur['close'] - cur['open'])
            if move_size < (cur['atr'] * atr_mult):
                continue

        if is_bull_move and ob['close'] < ob['open']:
            swept_liq = ob['low'] < ob_prev['low']
            if swept_liq:
                df.iloc[i - 2, df.columns.get_loc('ob_type')] = 1
                df.iloc[i - 2, df.columns.get_loc('ob_top')] = ob['high']
                df.iloc[i - 2, df.columns.get_loc('ob_bottom')] = ob['low']

        elif is_bear_move and ob['close'] > ob['open']:
            swept_liq = ob['high'] > ob_prev['high']
            if swept_liq:
                df.iloc[i - 2, df.columns.get_loc('ob_type')] = -1
                df.iloc[i - 2, df.columns.get_loc('ob_top')] = ob['high']
                df.iloc[i - 2, df.columns.get_loc('ob_bottom')] = ob['low']

    # -------------------
    # Breaker block detection
    # -------------------
    breakers = []
    obs = df[df['ob_type'] != 0]

    for idx in obs.index:
        loc = df.index.get_loc(idx)
        ob_type = obs.loc[idx, 'ob_type']
        ob_top = obs.loc[idx, 'ob_top']
        ob_bottom = obs.loc[idx, 'ob_bottom']

        # Step 1: Liquidity sweep
        sweep_found = False
        sweep_idx = None
        for j in range(loc + 1, min(len(df), loc + 1 + sweep_lookahead)):
            bar = df.iloc[j]
            if ob_type == 1 and bar['low'] < ob_bottom:  # Bullish OB swept below
                sweep_found = True
                sweep_idx = j
                break
            elif ob_type == -1 and bar['high'] > ob_top:  # Bearish OB swept above
                sweep_found = True
                sweep_idx = j
                break

        if not sweep_found:
            continue

        # Step 2: Opposite structure break
        for k in range(sweep_idx + 1, min(len(df), sweep_idx + 1 + break_lookahead)):
            bar = df.iloc[k]
            if ob_type == 1:
                recent_high = df['high'].iloc[max(0, k - 10):k].max()
                if bar['close'] > recent_high:
                    breakers.append({
                        'ob_index': idx,
                        'ob_type': ob_type,
                        'ob_top': ob_top,
                        'ob_bottom': ob_bottom,
                        'breaker_detected_at': df.index[k]
                    })
                    break
            else:
                recent_low = df['low'].iloc[max(0, k - 10):k].min()
                if bar['close'] < recent_low:
                    breakers.append({
                        'ob_index': idx,
                        'ob_type': ob_type,
                        'ob_top': ob_top,
                        'ob_bottom': ob_bottom,
                        'breaker_detected_at': df.index[k]
                    })
                    break

    return pd.DataFrame(breakers)



In [ ]:
# Htf Poi mitigation
def find_htf_poi_with_mitigation(
    htf_data,
    min_bars=10,
    min_penetration=0.0,   # 0.0 = any touch, 1.0 = full body penetration
    return_all=False,      # If False, returns latest bullish/bearish only
    use_body_only=False    # If True, checks only candle body for mitigation
):
    """
    Identify high-timeframe unmitigated order blocks (POIs) with more control.

    Args:
        htf_data (pd.DataFrame): Must contain open, high, low, close.
        min_bars (int): Minimum bars required to process.
        min_penetration (float): Min % penetration into OB for mitigation (0-1).
        return_all (bool): Return all unmitigated OBs if True.
        use_body_only (bool): Check only candle body for mitigation.

    Returns:
        list of dicts with OB details.
    """
    if htf_data is None or len(htf_data) < min_bars:
        return []

    # Ensure OBs exist
    htf_data = identify_order_blocks(htf_data.copy())
    if 'ob_type' not in htf_data.columns:
        return []

    valid_obs = htf_data[htf_data['ob_type'] != 0].copy()
    if valid_obs.empty:
        return []

    valid_obs['mitigated'] = False

    # Vectorized mitigation check
    for idx in valid_obs.index:
        try:
            loc = htf_data.index.get_loc(idx)
        except KeyError:
            continue

        ob_top = valid_obs.at[idx, 'ob_top']
        ob_bottom = valid_obs.at[idx, 'ob_bottom']

        # Get the future candles after the OB
        future = htf_data.iloc[loc+1:]

        if use_body_only:
            future_highs = future['close'].where(future['close'] > future['open'], future['open'])
            future_lows = future['close'].where(future['close'] < future['open'], future['open'])
        else:
            future_highs = future['high']
            future_lows = future['low']

        # Calculate penetration
        if valid_obs.at[idx, 'ob_type'] == 1:  # Bullish OB
            penetration = (ob_top - future_lows) / (ob_top - ob_bottom)
            if (penetration >= min_penetration).any():
                valid_obs.at[idx, 'mitigated'] = True
        else:  # Bearish OB
            penetration = (future_highs - ob_bottom) / (ob_top - ob_bottom)
            if (penetration >= min_penetration).any():
                valid_obs.at[idx, 'mitigated'] = True

    # Keep unmitigated OBs
    unmit = valid_obs[~valid_obs['mitigated']].copy()

    # Add metadata
    unmit['duration_bars'] = len(htf_data) - htf_data.index.get_indexer(unmit.index)

    if return_all:
        return unmit.to_dict('records')

    pois = []
    if not unmit[unmit['ob_type'] == 1].empty:
        pois.append(unmit[unmit['ob_type'] == 1].iloc[-1].to_dict())
    if not unmit[unmit['ob_type'] == -1].empty:
        pois.append(unmit[unmit['ob_type'] == -1].iloc[-1].to_dict())

    return pois


## HTF Analysis


In [ ]:
def find_valid_pullback_idm(
    df,
    swing_candle_index,
    lookback_limit=50,
    is_bullish_trend=True,
    htf_bias=None,
    use_swing_points=True,
    min_atr_mult=0.5,
    use_bodies=False,
    active_sessions=None  # e.g., [('09:30','16:00'), ('20:00','23:00')]
):
    """
    Find a valid pullback IDM level with improved filtering.

    Parameters:
    - df: DataFrame with 'high', 'low', 'open', 'close', 'atr', optional 'is_swing_high/low'.
    - swing_candle_index: Index of the swing point to start searching from.
    - lookback_limit: Max candles to look back.
    - is_bullish_trend: True if uptrend, False if downtrend.
    - htf_bias: 'bullish', 'bearish', or None → filters to match HTF direction.
    - use_swing_points: If True, only consider pullbacks at swing points.
    - min_atr_mult: Minimum ATR multiple for pullback displacement.
    - use_bodies: If True, use candle bodies instead of wicks for detection.
    - active_sessions: List of (start_time, end_time) tuples in HH:MM format to filter valid pullbacks.

    Returns:
    - pullback_level, pullback_index
    """
    try:
        swing_loc = df.index.get_loc(swing_candle_index)
    except KeyError:
        return np.nan, None
    if swing_loc < 1:
        return np.nan, None

    # Check HTF bias filter
    if htf_bias is not None:
        if htf_bias == 'bullish' and not is_bullish_trend:
            return np.nan, None
        if htf_bias == 'bearish' and is_bullish_trend:
            return np.nan, None

    limit_loc = max(0, swing_loc - lookback_limit)

    for i in range(swing_loc - 1, limit_loc - 1, -1):
        cur = df.iloc[i]
        prev = df.iloc[i - 1]

        # Optional: filter by active session
        if active_sessions:
            candle_time = pd.to_datetime(df.index[i]).time()
            if not any(start <= candle_time.strftime("%H:%M") <= end for start, end in active_sessions):
                continue

        # Swing point filter
        if use_swing_points:
            if is_bullish_trend and not cur.get('is_swing_low', False):
                continue
            if not is_bullish_trend and not cur.get('is_swing_high', False):
                continue

        # ATR filter
        if 'atr' in df.columns:
            move_size = abs(cur['close'] - prev['close'])
            if move_size < (cur['atr'] * min_atr_mult):
                continue

        # Wick/body detection
        if is_bullish_trend:
            val = cur['low'] if not use_bodies else min(cur['open'], cur['close'])
            prev_val = prev['low'] if not use_bodies else min(prev['open'], prev['close'])
            if val < prev_val:
                return val, df.index[i]
        else:
            val = cur['high'] if not use_bodies else max(cur['open'], cur['close'])
            prev_val = prev['high'] if not use_bodies else max(prev['open'], prev['close'])
            if val > prev_val:
                return val, df.index[i]

    return np.nan, None

def update_market_structure(
    df,
    market_state,
    swing_order=3,
    bos_break_type='close',  # 'close' or 'wick'
    min_bos_displacement=0.0,  # in price units or ATR multiples
    check_liquidity_sweep=False,
    htf_bias=None,
    plot=False
):
    """
    Update market structure with BOS/CHOCH & IDM tracking.

    Parameters:
    - df: DataFrame with OHLC + swing points + ATR + optional liquidity markers.
    - market_state: dict for persistent HTF state (passed in/out, no globals).
    - swing_order: Swing point order for detection.
    - bos_break_type: 'close' = close must break, 'wick' = high/low break.
    - min_bos_displacement: Minimum displacement to count as BOS.
    - check_liquidity_sweep: Require liquidity sweep before BOS.
    - htf_bias: 'bullish', 'bearish', or None to filter.
    - plot: If True, annotate df with rectangles/labels.

    Returns:
    - df: Updated DataFrame with BOS/CHOCH columns.
    - market_state: Updated state dict.
    - events: List of BOS/CHOCH events.
    """
    events = []

    if df is None or len(df) < swing_order * 2 + 1:
        return df, market_state, events

    df = identify_swing_points(df.copy(), order=swing_order)

    swing_highs = df[df['is_swing_high']]
    swing_lows = df[df['is_swing_low']]
    if swing_highs.empty or swing_lows.empty:
        return df, market_state, events

    last_sh_index = swing_highs.index[-1]
    last_sl_index = swing_lows.index[-1]
    last_sh_val = swing_highs['high'].iloc[-1]
    last_sl_val = swing_lows['low'].iloc[-1]

    # Determine trend context
    if last_sh_index > last_sl_index:
        new_idm_level, new_idm_time_index = find_valid_pullback_idm(
            df, last_sh_index, 50, True, htf_bias=htf_bias
        )
        idm_trend_context = 1
    else:
        new_idm_level, new_idm_time_index = find_valid_pullback_idm(
            df, last_sl_index, 50, False, htf_bias=htf_bias
        )
        idm_trend_context = -1

    # Store IDM if new
    if new_idm_time_index is not None and (
        market_state["idm_time_index"] is None or new_idm_time_index > market_state["idm_time_index"]
    ):
        market_state["idm_level"] = new_idm_level
        market_state["idm_time_index"] = new_idm_time_index
        market_state["idm_taken"] = False

    # Check IDM taken
    if not market_state["idm_taken"] and market_state["idm_time_index"] is not None:
        after = df[df.index > market_state["idm_time_index"]]
        if not after.empty:
            if idm_trend_context == 1 and (after['low'] < market_state["idm_level"]).any():
                market_state["idm_taken"] = True
            elif idm_trend_context == -1 and (after['high'] > market_state["idm_level"]).any():
                market_state["idm_taken"] = True

    # BOS/CHOCH detection
    df['BOS'] = 0
    df['CHOCH'] = 0
    start_check_index = market_state.get("last_bos_choch_time_index", df.index[0])
    candles_to_check = df[df.index > start_check_index]

    for current_index, current_candle in candles_to_check.iterrows():
        for direction in [1, -1]:
            if direction == 1 and not np.isnan(market_state.get("last_confirmed_high", np.nan)):
                trigger_price = (
                    market_state["last_confirmed_high"] if bos_break_type == 'wick' else current_candle['close']
                )
                if trigger_price > market_state["last_confirmed_high"]:
                    if check_liquidity_sweep and not current_candle.get('liq_sweep_high', False):
                        continue
                    events.append((current_index, 'BOS' if market_state["trend"] == 1 else 'CHOCH', direction))
                    df.loc[current_index, 'BOS' if market_state["trend"] == 1 else 'CHOCH'] = direction
                    market_state["trend"] = 1
                    market_state.update({
                        "last_confirmed_low": market_state["last_confirmed_high"],
                        "last_low_index": market_state["last_high_index"],
                        "last_confirmed_high": np.nan,
                        "last_high_index": None,
                        "idm_taken": False,
                        "idm_level": np.nan,
                        "idm_time_index": None,
                        "last_bos_choch_time_index": current_index
                    })
            elif direction == -1 and not np.isnan(market_state.get("last_confirmed_low", np.nan)):
                trigger_price = (
                    market_state["last_confirmed_low"] if bos_break_type == 'wick' else current_candle['close']
                )
                if trigger_price < market_state["last_confirmed_low"]:
                    if check_liquidity_sweep and not current_candle.get('liq_sweep_low', False):
                        continue
                    events.append((current_index, 'BOS' if market_state["trend"] == -1 else 'CHOCH', direction))
                    df.loc[current_index, 'BOS' if market_state["trend"] == -1 else 'CHOCH'] = direction
                    market_state["trend"] = -1
                    market_state.update({
                        "last_confirmed_high": market_state["last_confirmed_low"],
                        "last_high_index": market_state["last_low_index"],
                        "last_confirmed_low": np.nan,
                        "last_low_index": None,
                        "idm_taken": False,
                        "idm_level": np.nan,
                        "idm_time_index": None,
                        "last_bos_choch_time_index": current_index
                    })

    if plot:
        # Optional: plotting logic for BOS/CHOCH + IDM
        pass

    return df, market_state, events


## LTF Detection

In [ ]:
def map_ltf_structure(ltf_df, pct=0.004, use_atr=False, atr_period=14):
    """
    Map LTF market structure using zigzag-based swing highs/lows.

    Parameters:
        ltf_df (pd.DataFrame): OHLC dataframe with datetime index.
        pct (float): Zigzag percentage threshold for swing detection.
        use_atr (bool): If True, ATR is used for dynamic thresholds.
        atr_period (int): ATR period if use_atr=True.

    Returns:
        pd.DataFrame: Original DF with 'ltf_bos' and 'ltf_choch' columns.
    """
    # Copy to avoid modifying original
    ltf_df = ltf_df.copy()
    ltf_df['ltf_bos'] = 0
    ltf_df['ltf_choch'] = 0

    # --- Detect swing highs/lows ---
    pivots = zigzag_pivots_from_df(
        ltf_df, pct=pct, use_atr=use_atr, atr_period=atr_period
    )

    swing_highs = pd.DataFrame(
        [(ts, price) for ts, price, typ in pivots if typ == 'high'],
        columns=['ts', 'price']
    ).set_index('ts')

    swing_lows = pd.DataFrame(
        [(ts, price) for ts, price, typ in pivots if typ == 'low'],
        columns=['ts', 'price']
    ).set_index('ts')

    last_sh = swing_highs['price'].iloc[-1] if not swing_highs.empty else np.nan
    last_sl = swing_lows['price'].iloc[-1] if not swing_lows.empty else np.nan
    trend = 0  # 1 = bullish, -1 = bearish

    # --- Trend initialization ---
    if not swing_highs.empty and not swing_lows.empty:
        if swing_highs.index[-1] > swing_lows.index[-1] and len(swing_highs) > 1:
            if last_sh > swing_highs['price'].iloc[-2]:
                if len(swing_lows) > 1 and last_sl > swing_lows['price'].iloc[-2]:
                    trend = 1
        elif swing_lows.index[-1] > swing_highs.index[-1] and len(swing_lows) > 1:
            if last_sl < swing_lows['price'].iloc[-2]:
                if len(swing_highs) > 1 and last_sh < swing_highs['price'].iloc[-2]:
                    trend = -1

    # --- Loop through bars ---
    for i in range(1, len(ltf_df)):
        idx = ltf_df.index[i]
        close = ltf_df['close'].iloc[i]

        # Bullish break
        if not np.isnan(last_sh) and close > last_sh:
            if trend == 1:
                ltf_df.loc[idx, 'ltf_bos'] = 1
            else:
                ltf_df.loc[idx, 'ltf_choch'] = 1
            trend = 1
            # Update last swing low
            lows_before = swing_lows[swing_lows.index < idx]
            if not lows_before.empty:
                last_sl = lows_before['price'].iloc[-1]
            last_sh = ltf_df['high'].iloc[i]

        # Bearish break
        elif not np.isnan(last_sl) and close < last_sl:
            if trend == -1:
                ltf_df.loc[idx, 'ltf_bos'] = -1
            else:
                ltf_df.loc[idx, 'ltf_choch'] = -1
            trend = -1
            # Update last swing high
            highs_before = swing_highs[swing_highs.index < idx]
            if not highs_before.empty:
                last_sh = highs_before['price'].iloc[-1]
            last_sl = ltf_df['low'].iloc[i]

    return ltf_df





def find_ltf_idm_after_choch(
    ltf_df, choch_index, is_bullish_choch,
    look_ahead_bars=10, min_displacement=0.0, break_check_bars=5
):
    """
    Finds the first valid IDM level after a CHOCH in LTF data.

    Parameters:
        ltf_df: DataFrame with 'high', 'low' columns.
        choch_index: Index of CHOCH candle.
        is_bullish_choch: True for bullish CHOCH, False for bearish.
        look_ahead_bars: How many candles ahead to search for IDM.
        min_displacement: Minimum price move to qualify as IDM.
        break_check_bars: How many candles to check for IDM being taken.

    Returns:
        (idm_level, idm_time_index, idm_taken)
    """
    try:
        choch_loc = ltf_df.index.get_loc(choch_index)
    except KeyError:
        return np.nan, None, False

    idm_level = np.nan
    idm_time_index = None
    idm_taken = False

    search_end = min(len(ltf_df), choch_loc + look_ahead_bars + 1)

    for i in range(choch_loc + 1, search_end):
        cur = ltf_df.iloc[i]
        prev = ltf_df.iloc[i - 1]

        if is_bullish_choch and cur['low'] < prev['low'] - min_displacement:
            idm_level = cur['low']
            idm_time_index = ltf_df.index[i]
            break
        elif not is_bullish_choch and cur['high'] > prev['high'] + min_displacement:
            idm_level = cur['high']
            idm_time_index = ltf_df.index[i]
            break

    # Check if IDM is taken within allowed bars
    if idm_time_index:
        end_check = min(len(ltf_df), ltf_df.index.get_loc(idm_time_index) + break_check_bars + 1)
        after_idm = ltf_df.iloc[ltf_df.index.get_loc(idm_time_index) + 1:end_check]
        if is_bullish_choch and (after_idm['low'] < idm_level).any():
            idm_taken = True
        elif not is_bullish_choch and (after_idm['high'] > idm_level).any():
            idm_taken = True

    return idm_level, idm_time_index, idm_taken



def find_entry_zone_after_idm(ltf_df, idm_taken_index, is_bullish_entry):
    """
    Find the first valid Order Block (OB) or Fair Value Gap (FVG)
    after an IDM has been taken out.

    ltf_df: DataFrame with OHLC and liquidity structure columns.
    idm_taken_index: index (timestamp) where IDM was taken.
    is_bullish_entry: True for bullish setups, False for bearish.

    Returns:
        pd.Series of the chosen OB/FVG candle, or None if not found.
    """
    try:
        idm_loc = ltf_df.index.get_loc(idm_taken_index)
    except KeyError:
        return None

    # Ensure OB & FVG columns are present
    ltf_df = identify_order_blocks(ltf_df.copy())

    for i in range(idm_loc, len(ltf_df)):
        candle = ltf_df.iloc[i]
        prev_candle = ltf_df.iloc[i-1] if i > 0 else None

        # ----------- Bullish case -----------
        if is_bullish_entry:
            # ✅ Order Block check
            if candle['ob_type'] == 1:
                if i + 1 < len(ltf_df) and not (ltf_df['low'].iloc[i+1:] < candle['ob_bottom']).any():
                    return candle
            # ✅ Fair Value Gap check
            if prev_candle is not None and prev_candle['fvg_type'] == 1:
                if not (ltf_df['low'].iloc[i:] < prev_candle['fvg_bottom']).any():
                    return prev_candle

        # ----------- Bearish case -----------
        else:
            # ✅ Order Block check
            if candle['ob_type'] == -1:
                if i + 1 < len(ltf_df) and not (ltf_df['high'].iloc[i+1:] > candle['ob_top']).any():
                    return candle
            # ✅ Fair Value Gap check
            if prev_candle is not None and prev_candle['fvg_type'] == -1:
                if not (ltf_df['high'].iloc[i:] > prev_candle['fvg_top']).any():
                    return prev_candle

    # No valid zone found
    return None



## Equilibrium

In [ ]:
def compute_equilibrium_between_swings(df, high_index, low_index):
    """
    Returns the midpoint price between swing high and swing low (equilibrium).
    Accepts indices (timestamps) for the swing high and low.
    """
    try:
        high_val = df.loc[high_index]['high']
        low_val = df.loc[low_index]['low']
    except Exception:
        return None
    return (high_val + low_val) / 2.0

def is_entry_in_discount_or_premium(entry_price, equilibrium_price, signal_type):
    """
    For BUY require entry < equilibrium (discount).
    For SELL require entry > equilibrium (premium).
    """
    if equilibrium_price is None:
        return True
    if signal_type == 'BUY':
        return entry_price < equilibrium_price
    else:
        return entry_price > equilibrium_price

# LTF ifvg and breakers

In [ ]:
# LTF Ifvg and breaker blocks

def check_ltf_confirmation_advanced(ltf_data, poi, signal_type):
    """
    Returns (confirmation_type, entry_price, sl_price, confluences_list)
    New confluences: 'sweep', 'choch_idm', 'flip', 'ifvg', 'breaker', 'ob'
    Equilibrium filter applied when possible.
    """
    if ltf_data is None or len(ltf_data) < 30:
        return None, None, None, []
    poi_top = poi['ob_top']; poi_bottom = poi['ob_bottom']
    poi_size = poi_top - poi_bottom if (not pd.isna(poi_top) and not pd.isna(poi_bottom)) else 0.0

    # Find mitigation start
    mitigation_start_index = None; mitigation_loc = -1
    check_range = min(60, len(ltf_data))
    for i in range(len(ltf_data)-1, len(ltf_data)-check_range-1, -1):
        candle = ltf_data.iloc[i]
        if not (pd.isna(candle['low']) or pd.isna(candle['high'])):
            if max(candle['low'], poi_bottom) <= min(candle['high'], poi_top):
                mitigation_start_index = ltf_data.index[i]; mitigation_loc = i
                break
    if mitigation_start_index is None:
        return None, None, None, []

    ltf_since = ltf_data.iloc[mitigation_loc:].copy()
    if len(ltf_since) < 5:
        return None, None, None, []

    # compute ATR for buffer
    atr_series = atr(ltf_data, 14)
    atr_since = atr_series.loc[ltf_since.index] if atr_series is not None else None

    confluences = []
    entry_price = None; sl_price = None; confirmation = None

    # Sweep detection (improved)
    sweep_found = False; sweep_entry=None; sweep_sl=None
    lookback_n = 8
    recent_lows_min = ltf_since['low'].iloc[max(0, len(ltf_since)-lookback_n):].min()
    recent_highs_max = ltf_since['high'].iloc[max(0, len(ltf_since)-lookback_n):].max()
    for i in range(1, len(ltf_since)):
        cur = ltf_since.iloc[i]; prev = ltf_since.iloc[i-1]
        in_poi = max(cur['low'], poi_bottom) <= min(cur['high'], poi_top)
        if signal_type == 'BUY' and in_poi:
            if cur['low'] < recent_lows_min:
                if cur['close'] > cur['open'] and cur['close'] > (cur['open'] + cur['low'])/2:
                    sweep_found=True; sweep_entry = cur['close']; sweep_sl = cur['low'] - max(abs(poi_size)*SL_BUFFER_PCT, 0.00001); break
        if signal_type == 'SELL' and in_poi:
            if cur['high'] > recent_highs_max:
                if cur['close'] < cur['open'] and cur['close'] < (cur['open'] + cur['high'])/2:
                    sweep_found=True; sweep_entry = cur['close']; sweep_sl = cur['high'] + max(abs(poi_size)*SL_BUFFER_PCT, 0.00001); break
    if sweep_found: confluences.append('sweep')

    # CHOCH + IDM detection (like earlier)
    ltf_struct_df = map_ltf_structure(ltf_since)
    choch_signals = ltf_struct_df[ltf_struct_df['ltf_choch'] != 0]
    choch_found=False; choch_entry=None; choch_sl=None
    if not choch_signals.empty:
        first_choch = choch_signals.iloc[0]; choch_index = choch_signals.index[0]; choch_type = int(first_choch['ltf_choch'])
        if (signal_type=='BUY' and choch_type==1) or (signal_type=='SELL' and choch_type==-1):
            if atr_since is not None and choch_index in atr_since.index:
                choch_atr = atr_since.loc[choch_index]
            else:
                choch_atr = max(0.0005, (ltf_since['high']-ltf_since['low']).median())
            if choch_type == 1:
                broken_level = first_choch['high'] if 'high' in first_choch.index else None
                buffer = choch_atr * CHOCH_ATR_BUFFER
                if broken_level is not None and (first_choch['close'] - broken_level) > buffer:
                    idm_level, idm_time_index, idm_taken = find_ltf_idm_after_choch(ltf_struct_df, choch_index, True)
                    if idm_time_index is not None and idm_taken:
                        entry_zone = find_entry_zone_after_idm(ltf_struct_df, idm_time_index, True)
                        if entry_zone is not None:
                            choch_found=True
                            if not pd.isna(entry_zone['ob_type']):
                                choch_entry = entry_zone['ob_top']; choch_sl = entry_zone['ob_bottom'] - abs(entry_zone['ob_top']-entry_zone['ob_bottom'])*SL_BUFFER_PCT
                            elif not pd.isna(entry_zone['fvg_type']):
                                choch_entry = entry_zone['fvg_top']; choch_sl = entry_zone['fvg_bottom'] - abs(entry_zone['fvg_top']-entry_zone['fvg_bottom'])*SL_BUFFER_PCT
            else:
                broken_level = first_choch['low'] if 'low' in first_choch.index else None
                buffer = choch_atr * CHOCH_ATR_BUFFER
                if broken_level is not None and (broken_level - first_choch['close']) > buffer:
                    idm_level, idm_time_index, idm_taken = find_ltf_idm_after_choch(ltf_struct_df, choch_index, False)
                    if idm_time_index is not None and idm_taken:
                        entry_zone = find_entry_zone_after_idm(ltf_struct_df, idm_time_index, False)
                        if entry_zone is not None:
                            choch_found=True
                            if not pd.isna(entry_zone['ob_type']):
                                choch_entry = entry_zone['ob_bottom']; choch_sl = entry_zone['ob_top'] + abs(entry_zone['ob_top']-entry_zone['ob_bottom'])*SL_BUFFER_PCT
                            elif not pd.isna(entry_zone['fvg_type']):
                                choch_entry = entry_zone['fvg_bottom']; choch_sl = entry_zone['fvg_top'] + abs(entry_zone['fvg_top']-entry_zone['fvg_bottom'])*SL_BUFFER_PCT
    if choch_found: confluences.append('choch_idm')

    # Flip detection
    flip_found=False; flip_entry=None; flip_sl=None
    ltf_obs = identify_order_blocks(ltf_since.copy())
    for i in range(1, len(ltf_obs)):
        ob_prev = ltf_obs.iloc[i-1]
        if signal_type=='BUY':
            if ob_prev['ob_type']==-1 and max(ob_prev['low'], poi_bottom) <= min(ob_prev['high'], poi_top):
                if (ltf_obs['close'].iloc[i:] > ob_prev['ob_top']).any():
                    subsequent_bull = ltf_obs[(ltf_obs.index > ltf_obs.index[i-1]) & (ltf_obs['ob_type']==1)]
                    if not subsequent_bull.empty:
                        flip_found=True
                        flip_entry = subsequent_bull.iloc[0]['ob_top']
                        flip_sl = subsequent_bull.iloc[0]['ob_bottom'] - abs(subsequent_bull.iloc[0]['ob_top']-subsequent_bull.iloc[0]['ob_bottom'])*SL_BUFFER_PCT
                        break
        else:
            if ob_prev['ob_type']==1 and max(ob_prev['low'], poi_bottom) <= min(ob_prev['high'], poi_top):
                if (ltf_obs['close'].iloc[i:] < ob_prev['ob_bottom']).any():
                    subsequent_bear = ltf_obs[(ltf_obs.index > ltf_obs.index[i-1]) & (ltf_obs['ob_type']==-1)]
                    if not subsequent_bear.empty:
                        flip_found=True
                        flip_entry = subsequent_bear.iloc[0]['ob_bottom']
                        flip_sl = subsequent_bear.iloc[0]['ob_top'] + abs(subsequent_bear.iloc[0]['ob_top']-subsequent_bear.iloc[0]['ob_bottom'])*SL_BUFFER_PCT
                        break
    if flip_found: confluences.append('flip')

    # iFVG detection: use HTF or LTF unmitigated FVGs created before the last HTF break time (if available)
    ifvg_found=False; ifvg_entry=None; ifvg_sl=None
    # Use the global HTF last_bos_choch_time_index as possible reference
    ref_break_time = market_state_htf.get("last_bos_choch_time_index", None)
    i_fvgs = identify_ifvg(ltf_data, reference_break_time=ref_break_time)
    # choose iFVG inside POI (or close to it)
    if not i_fvgs.empty:
        for idx,row in i_fvgs.iterrows():
            # check if iFVG is near POI (overlapping or adjacent)
            if not (pd.isna(row['fvg_bottom']) or pd.isna(row['fvg_top'])):
                overlap = max(row['fvg_bottom'], poi_bottom) <= min(row['fvg_top'], poi_top)
                if overlap:
                    if row['fvg_type'] == 1 and signal_type == 'BUY':
                        ifvg_found=True; ifvg_entry = row['fvg_top']; ifvg_sl = row['fvg_bottom'] - abs(row['fvg_top']-row['fvg_bottom'])*SL_BUFFER_PCT; break
                    if row['fvg_type'] == -1 and signal_type == 'SELL':
                        ifvg_found=True; ifvg_entry = row['fvg_bottom']; ifvg_sl = row['fvg_top'] + abs(row['fvg_top']-row['fvg_bottom'])*SL_BUFFER_PCT; break
    if ifvg_found: confluences.append('ifvg')

    # Breaker block detection
    breaker_found=False; breaker_entry=None; breaker_sl=None
    breakers = identify_breaker_blocks(ltf_data)
    if breakers:
        # pick most recent breaker inside or near POI
        for b in reversed(breakers):
            b_top = b['ob_top']; b_bottom = b['ob_bottom']
            overlap = max(b_bottom, poi_bottom) <= min(b_top, poi_top)
            if overlap:
                if b['ob_type'] == 1 and signal_type == 'BUY':
                    breaker_found=True; breaker_entry = b_top; breaker_sl = b_bottom - abs(b_top-b_bottom)*SL_BUFFER_PCT; break
                if b['ob_type'] == -1 and signal_type == 'SELL':
                    breaker_found=True; breaker_entry = b_bottom; breaker_sl = b_top + abs(b_top-b_bottom)*SL_BUFFER_PCT; break
    if breaker_found: confluences.append('breaker')

    # OB present in entry zone is already captured via choch logic but we still add if entry_zone was OB
    # Compose final confluences set
    unique_confs = list(set(confluences))

    # require MIN_CONFLUENCES
    if len(unique_confs) < MIN_CONFLUENCES:
        return None, None, None, unique_confs

    # Equilibrium check: try to compute between last confirmed HTF swings (if available)
    equilibrium_price = None
    try:
        high_idx = market_state_htf.get("last_high_index", None)
        low_idx = market_state_htf.get("last_low_index", None)
        # Use HTF cache if exists
        htf_df = _cache.get("htf_df", None)
        if high_idx is not None and low_idx is not None and htf_df is not None:
            equilibrium_price = compute_equilibrium_between_swings(htf_df, high_idx, low_idx)
    except Exception:
        equilibrium_price = None

    # Choose final priority: prefer choch_idm + (ifvg|breaker|sweep) etc.
    # Candidate mapping
    candidate = None
    candidate_sl = None
    candidate_conf = None
    priority = ['choch_idm','ifvg','breaker','sweep','flip']
    # if choch_idm exists, try to use its entry
    if 'choch_idm' in unique_confs and choch_entry is not None and choch_sl is not None:
        candidate, candidate_sl, candidate_conf = choch_entry, choch_sl, 'choch_idm'
    elif 'ifvg' in unique_confs and ifvg_entry is not None and ifvg_sl is not None:
        candidate, candidate_sl, candidate_conf = ifvg_entry, ifvg_sl, 'ifvg'
    elif 'breaker' in unique_confs and breaker_entry is not None and breaker_sl is not None:
        candidate, candidate_sl, candidate_conf = breaker_entry, breaker_sl, 'breaker'
    elif 'sweep' in unique_confs and sweep_entry is not None and sweep_sl is not None:
        candidate, candidate_sl, candidate_conf = sweep_entry, sweep_sl, 'sweep'
    elif 'flip' in unique_confs and flip_entry is not None and flip_sl is not None:
        candidate, candidate_sl, candidate_conf = flip_entry, flip_sl, 'flip'
    else:
        # fallback pick whichever exists
        for e,s in [('choch',choch_entry),('ifvg',ifvg_entry),('breaker',breaker_entry),('sweep',sweep_entry),('flip',flip_entry)]:
            if s is not None:
                candidate = s; candidate_conf = e; break

    if candidate is None:
        return None, None, None, unique_confs

    # Equilibrium filter: ensure entry is on discount/premium side
    if equilibrium_price is not None:
        if not is_entry_in_discount_or_premium(candidate, equilibrium_price, signal_type):
            logging.info(f"Rejected entry by Equilibrium filter: entry {candidate} eq {equilibrium_price} type {signal_type}")
            return None, None, None, unique_confs

    # final sanity checks
    if signal_type == 'BUY' and candidate_sl >= candidate:
        logging.warning("Invalid SL for BUY.")
        return None, None, None, unique_confs
    if signal_type == 'SELL' and candidate_sl <= candidate:
        logging.warning("Invalid SL for SELL.")
        return None, None, None, unique_confs

    return candidate_conf, candidate, candidate_sl, unique_confs

## Sesssion Times

In [ ]:
# ---------------- Utility: session filter ----------------
def is_within_allowed_session_utc(now_utc):
    s = now_utc.time()
    for start_str, end_str in ALLOWED_SESSIONS_UTC:
        start = datetime.strptime(start_str, "%H:%M").time()
        end = datetime.strptime(end_str, "%H:%M").time()
        if start <= s <= end:
            return True
    return False

## Signal Genaration

In [ ]:
# ---------------- Signal generation ----------------
def generate_signals_advanced(symbol, htf_tf, ltf_tf):
    global market_state_htf
    htf_data = get_rates(symbol, htf_tf, LOOKBACK_CANDLES_HTF)
    ltf_data = get_rates(symbol, ltf_tf, LOOKBACK_CANDLES_LTF)
    if htf_data is None or ltf_data is None:
        logging.error("Insufficient market data.")
        return None, None, None, None
    _cache['htf_df'] = htf_data; _cache['ltf_df'] = ltf_data
    htf_data = update_market_structure(htf_data)
    htf_pois = find_htf_poi_with_mitigation(htf_data)
    if not htf_pois: return None, None, None, None
    # filter POIs aligned with HTF trend
    aligned = []
    for poi in htf_pois:
        poi_type = int(poi['ob_type'])
        if market_state_htf['trend'] == 0:
            aligned.append(poi)
        else:
            if (market_state_htf['trend'] == 1 and poi_type == 1) or (market_state_htf['trend'] == -1 and poi_type == -1):
                aligned.append(poi)
    if not aligned:
        logging.info("No POIs aligned with HTF trend.")
        return None, None, None, None
    for poi in aligned:
        signal_type = 'BUY' if int(poi['ob_type'])==1 else 'SELL'
        conf_type, entry_price, sl_price, confs = check_ltf_confirmation_advanced(ltf_data, poi, signal_type)
        if conf_type is not None:
            logging.info(f"Signal {signal_type} via {conf_type}, confluences: {confs}, entry:{entry_price}, sl:{sl_price}")
            return signal_type, entry_price, sl_price, conf_type
    return None, None, None, None


## Backtesting

In [ ]:
ohlc_bt = df_1m.rename(columns={
    'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'
})

# Generate signals and store them in a DataFrame
# You might need to adjust parameters for generate_signals_advanced based on your needs
signal_list = []
for i in range(len(ohlc_bt)):
    # Pass a slice of the data up to the current point to the signal generation function
    current_data_htf = df_4h.iloc[:df_4h.index.get_loc(ohlc_bt.index[i]) + 1]
    current_data_ltf = df_1m.iloc[:i + 1]

    # Ensure there is enough data for signal generation
    if len(current_data_htf) > LOOKBACK_CANDLES_HTF and len(current_data_ltf) > LOOKBACK_CANDLES_LTF:
        signal_type, entry_price, sl_price, conf_type = generate_signals_advanced("XAUUSDm", "4h", "1m") # Use appropriate symbol and timeframes
        if signal_type:
            signal_list.append({
                'DateTime': ohlc_bt.index[i],
                'signal': signal_type,
                'entry': entry_price,
                'sl': sl_price,
                'tp': np.nan, # You need to add TP calculation logic
                'conf_type': conf_type
            })

signals_df = pd.DataFrame(signal_list).set_index('DateTime')


class SMCStrategy(Strategy):
    def init(self):
        # Ensure signals_df is accessible here
        self.signals = signals_df

    def next(self):
        dt = self.data.index[-1]
        if dt in self.signals.index:
            row = self.signals.loc[dt]
            if row['signal'] == 'BUY':
                # Check if entry price is met before placing order
                if self.data.Close[-1] <= row['entry']:
                     self.buy(sl=row['sl'], tp=row['tp'])
            elif row['signal'] == 'SELL':
                # Check if entry price is met before placing order
                if self.data.Close[-1] >= row['entry']:
                    self.sell(sl=row['sl'], tp=row['tp'])

# =========================================================
# 7️⃣ Run Backtest
# =========================================================
bt = Backtest(ohlc_bt, SMCStrategy, cash=10000, commission=0.0002, trade_on_close=True)
stats = bt.run()
print(stats)
bt.plot()

KeyError: Timestamp('2025-04-07 16:44:00')